# Use case proposed in [issue #78](https://github.com/bsorrentino/langgraph4j/issues/93) by [tansice](https://github.com/tansice)

## There is a null pointer issue in ToolExecutionRequestSerializer.

In [1]:
var userHomeDir = System.getProperty("user.home");
var localRespoUrl = "file://" + userHomeDir + "/.m2/repository/";
var langchain4jVersion = "1.0.1";
var langchain4jbeta = "1.0.1-beta6";
var langgraph4jVersion = "1.6-SNAPSHOT";

In [2]:
%%bash 
rm -rf \{userHomeDir}/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/langgraph4j

In [3]:
%dependency /add-repo local \{localRespoUrl} release|never snapshot|always
// %dependency /list-repos
%dependency /add org.slf4j:slf4j-jdk14:2.0.9
%dependency /add org.bsc.langgraph4j:langgraph4j-core:\{langgraph4jVersion}
%dependency /add org.bsc.langgraph4j:langgraph4j-langchain4j:\{langgraph4jVersion}
%dependency /add org.bsc.langgraph4j:langgraph4j-agent-executor:\{langgraph4jVersion}
%dependency /add dev.langchain4j:langchain4j:\{langchain4jVersion}
%dependency /add dev.langchain4j:langchain4j-ollama:\{langchain4jbeta}

%dependency /resolve

Repository local url: file:///Users/bsorrentino/.m2/repository/ added.
Adding dependency org.slf4j:slf4j-jdk14:2.0.9
Adding dependency org.bsc.langgraph4j:langgraph4j-core:1.6-SNAPSHOT
Adding dependency org.bsc.langgraph4j:langgraph4j-langchain4j:1.6-SNAPSHOT
Adding dependency org.bsc.langgraph4j:langgraph4j-agent-executor:1.6-SNAPSHOT
Adding dependency dev.langchain4j:langchain4j:1.0.1
Adding dependency dev.langchain4j:langchain4j-ollama:1.0.1-beta6
Solving dependencies
Resolved artifacts count: 55
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/slf4j/slf4j-jdk14/2.0.9/slf4j-jdk14-2.0.9.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/slf4j/slf4j-api/2.0.9/slf4j-api-2.0.9.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/langgraph4j/langgraph4j-core/1.6-SNAPSHOT/langgraph4j-core-1.6-SNAPSHOT.jar
Add to classpath: /Users/bsorrent

**Initialize Logger**

In [4]:
try( var file = new java.io.FileInputStream("./logging.properties")) {
    java.util.logging.LogManager.getLogManager().readConfiguration( file );
}

var log = org.slf4j.LoggerFactory.getLogger("default");


In [5]:
import dev.langchain4j.agent.tool.P;
import dev.langchain4j.agent.tool.Tool;
import dev.langchain4j.data.message.AiMessage;
import dev.langchain4j.data.message.ChatMessage;
import dev.langchain4j.data.message.UserMessage;
import dev.langchain4j.model.chat.request.ChatRequest;
import dev.langchain4j.model.chat.request.ChatRequestParameters;
import dev.langchain4j.model.ollama.OllamaStreamingChatModel;
import org.bsc.langgraph4j.action.AsyncEdgeAction;
import org.bsc.langgraph4j.action.AsyncNodeAction;
import org.bsc.langgraph4j.action.EdgeAction;
import org.bsc.langgraph4j.action.NodeAction;
import org.bsc.langgraph4j.langchain4j.generators.StreamingChatGenerator;
import org.bsc.langgraph4j.langchain4j.serializer.std.ChatMesssageSerializer;
import org.bsc.langgraph4j.langchain4j.serializer.std.ToolExecutionRequestSerializer;
import org.bsc.langgraph4j.langchain4j.tool.ToolNode;
import org.bsc.langgraph4j.prebuilt.MessagesState;
import org.bsc.langgraph4j.prebuilt.MessagesStateGraph;
import org.bsc.langgraph4j.serializer.std.ObjectStreamStateSerializer;
import org.slf4j.Logger;
import org.slf4j.LoggerFactory;

import java.util.Map;

import static org.bsc.langgraph4j.StateGraph.END;
import static org.bsc.langgraph4j.StateGraph.START;
import static org.bsc.langgraph4j.action.AsyncEdgeAction.edge_async;
import static org.bsc.langgraph4j.action.AsyncNodeAction.node_async;

static class SearchTool {
    @Tool("Use to surf the web, fetch current information, check the weather, and retrieve other information.")
    String execQuery(@P("The query to use in your search.") String query) {
        return "Cold, with a low of 13 degrees";
    }
}

void testToolsStreamingChat( String modelName ) throws Exception {
    // Setup streaming model
    var model = OllamaStreamingChatModel.builder()
            .baseUrl("http://localhost:11434")
            .temperature(0.5)
            .logRequests(true)
            .logResponses(true)
            .modelName( modelName )
            //.modelName("qwen2.5:7b")
            //.modelName("llama3.1:latest")
            .build();

    // Setup tools
    var tools = ToolNode.builder()
            .specification(new SearchTool())
            .build();

    // Setup serializers
    ObjectStreamStateSerializer<MessagesState<ChatMessage>> stateSerializer = new ObjectStreamStateSerializer<>(MessagesState::new);
    stateSerializer.mapper()
            .register(dev.langchain4j.agent.tool.ToolExecutionRequest.class, new ToolExecutionRequestSerializer())
            .register(ChatMessage.class, new ChatMesssageSerializer());

    // Define graph
    NodeAction<MessagesState<ChatMessage>> callModel = state -> {
        var generator = StreamingChatGenerator.<MessagesState<ChatMessage>>builder()
                .mapResult(response -> Map.of("messages", response.aiMessage()))
                .startingNode("agent")
                .startingState(state)
                .build();

        var parameters = ChatRequestParameters.builder()
                .toolSpecifications(tools.toolSpecifications())
                .build();
        var request = ChatRequest.builder()
                .messages(state.messages())
                .parameters(parameters)
                .build();

        model.chat(request, generator.handler());

        return Map.of("_streaming_messages", generator);
    };

    EdgeAction<MessagesState<ChatMessage>> routeMessage = state -> {
        var lastMessage = state.lastMessage()
                .orElseThrow(() -> new IllegalStateException("last message not found!"));

        if (lastMessage instanceof AiMessage message) {
            if (message.hasToolExecutionRequests()) {
                return "next";
            }
        }

        return "exit";
    };

    NodeAction<MessagesState<ChatMessage>> invokeTool = state -> {
        var lastMessage = state.lastMessage()
                .orElseThrow(() -> new IllegalStateException("last message not found!"));

        if (lastMessage instanceof AiMessage lastAiMessage) {
            var result = tools.execute(lastAiMessage.toolExecutionRequests(), null)
                    .orElseThrow(() -> new IllegalStateException("no tool found!"));

            return Map.of("messages", result);
        }

        throw new IllegalStateException("invalid last message");
    };

    var workflow = new MessagesStateGraph<>(stateSerializer)
            .addNode("agent", node_async(callModel))
            .addNode("tools", node_async(invokeTool))
            .addEdge(START, "agent")
            .addConditionalEdges("agent",
                    edge_async(routeMessage),
                    Map.of("next", "tools", "exit", END))
            .addEdge("tools", "agent");

    var app = workflow.compile();

    var output = app.stream(Map.of("messages", UserMessage.from("How is the weather in New York today?")));
    for (var out : output) {
        log.info("StreamingOutput: {}", out);
    }
}



In [6]:
testToolsStreamingChat("qwen2.5:7b");

START 
ToolExecutionRequest id is null! 
StreamingOutput: NodeOutput{node=__START__, state={messages=[UserMessage { name = null contents = [TextContent { text = "How is the weather in New York today?" }] }]}} 
ToolExecutionRequest id is null! 
execute: execQuery 
ToolExecutionRequest id is null! 
ToolExecutionResultMessage id is null! 
StreamingOutput: NodeOutput{node=agent, state={messages=[UserMessage { name = null contents = [TextContent { text = "How is the weather in New York today?" }] }, AiMessage { text = null toolExecutionRequests = [ToolExecutionRequest { id = null, name = "execQuery", arguments = "{
  "query" : "weather in New York today"
}" }] }]}} 
ToolExecutionRequest id is null! 
ToolExecutionResultMessage id is null! 
StreamingOutput: NodeOutput{node=tools, state={messages=[UserMessage { name = null contents = [TextContent { text = "How is the weather in New York today?" }] }, AiMessage { text = null toolExecutionRequests = [ToolExecutionRequest { id = null, name = "exe

In [7]:

testToolsStreamingChat("llama3.1:latest");

START 
ToolExecutionRequest id is null! 
StreamingOutput: NodeOutput{node=__START__, state={messages=[UserMessage { name = null contents = [TextContent { text = "How is the weather in New York today?" }] }]}} 
ToolExecutionRequest id is null! 
execute: execQuery 
ToolExecutionRequest id is null! 
ToolExecutionResultMessage id is null! 
StreamingOutput: NodeOutput{node=agent, state={messages=[UserMessage { name = null contents = [TextContent { text = "How is the weather in New York today?" }] }, AiMessage { text = null toolExecutionRequests = [ToolExecutionRequest { id = null, name = "execQuery", arguments = "{
  "query" : "New York weather today"
}" }] }]}} 
ToolExecutionRequest id is null! 
ToolExecutionResultMessage id is null! 
StreamingOutput: NodeOutput{node=tools, state={messages=[UserMessage { name = null contents = [TextContent { text = "How is the weather in New York today?" }] }, AiMessage { text = null toolExecutionRequests = [ToolExecutionRequest { id = null, name = "execQu